In [ ]:
!unzip /content/drive/MyDrive/pothole600_v3.zip

In [ ]:
import os
import cv2
import glob
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.optimizers import Adam
import plotly.graph_objects as go
from keras.layers import LeakyReLU
from plotly.subplots import make_subplots
from helper import DenseNet_UNET, DataGenerator, loss_function, accuracy_function, get_df


plt.style.use("ggplot")
%matplotlib inline

# Upload the models weights

In [ ]:
mask_model = DenseNet_UNET(64, LeakyReLU(0.2))
mask_model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy", tf.keras.metrics.BinaryIoU(), 
                  tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

mask_model.load_weights('/content/drive/MyDrive/smartathon/Label-MODEL-WITH-agumentation-ex1.h5')

74836368/74836368 [==============================] - 4s 0us/step


In [ ]:
depth_model = tf.keras.models.load_model('/content/drive/MyDrive/smartathon/GRAY_DEPTH-MODEL-WITH-nyu-depth-ex1.h5'
, custom_objects={'loss_function':loss_function, 'accuracy_function':accuracy_function})


---

---

# The hackathon dataset

In [ ]:
SA_potholes = glob.glob("/content/drive/MyDrive/smartathon/cropped_images/*")
SA_potholes_test = pd.DataFrame()
SA_potholes_test['image'] = SA_potholes
SA_potholes_test['mask'] = SA_potholes

There are 163 images from the hackathon dataset that has been unseen by any model before, neither the pothole classifier model nor the mask labelling model nor the depth estimation model. That's why I will try to predict the pothole mask on this subset for our hackathon dataset.

In [ ]:
SA_potholestest_generator = DataGenerator(SA_potholes_test, 'mask', batch_size=163, shuffle=False, dim=(320,320))
SA_potholestest_images, _ = next(iter(SA_potholestest_generator))
SA_potholestest_preds = mask_model.predict(np.array(SA_potholestest_images))

6/6 [==============================] - 255s 40s/step


The model took 255s to predict 163 images mask label, this means each image took around `1.5s`.

In [ ]:
cmap = "plasma_r"


for i in range(len(SA_potholestest_images)):
  plt.figure(figsize=(19,10))
  pred = SA_potholestest_preds[i]
  pred = np.squeeze(pred, axis=-1)
  plt.subplot(1,3,1)
  plt.axis("off")
  plt.imshow(pred, cmap=plt.get_cmap(cmap))


  plt.subplot(1,3,2)
  plt.axis("off")
  img1 = SA_potholestest_images[i]
  plt.imshow(img1)

  plt.show()

The model failed miserably in predicting the pothole mask for the hackathon dataset, and the reason behind that as I believe is the difference in the dataset's nature. The model was trained on a subset of the pothole600 dataset, where the potholes images are taken from a close distance and the dataset was collected using a ZED stereo camera so this will represent more accurate images for the pothole. To test if our mask labelling model will help us to make 3d representation for the pothole we can test the model on unseen dataset where the images collected from the same distance with the same camera.

---

# The pothole600 dataset


In [ ]:
pothole600_test = get_df('/content/testing')
pothole600_mask_generator = DataGenerator(pothole600_test, 'mask', batch_size=80, shuffle=False, dim=(320,320))
pothole600_mask_images,pothole600_mask_labels = next(iter(pothole600_mask_generator))
pothole600_preds = mask_model.predict(np.array(pothole600_mask_images))

3/3 [==============================] - 175s 51s/step


In [ ]:
def plot_image_mask(images, preds, labels):
  cmap = "plasma_r"
  for i in range(len(images)):
    plt.figure(figsize=(19,10))
    pred = preds[i]
    pred = np.squeeze(pred, axis=-1)
    plt.subplot(1,3,1)
    plt.axis("off")
    plt.imshow(pred, cmap=plt.get_cmap(cmap))

    plt.subplot(1,3,2)
    plt.axis("off")
    img = labels[i]
    img = np.squeeze(img, axis=-1)
    plt.imshow(img, cmap=plt.get_cmap(cmap))

    plt.subplot(1,3,3)
    plt.axis("off")
    img1 = images[i]
    plt.imshow(img1)

    plt.show()

In [19]:
plot_image_mask(pothole600_mask_images, pothole600_preds, pothole600_mask_labels)

In general the model works find to localize the pothole(s). We can see the predicted images or mask on the left or first column have more smoother edges than the true mask on the second column. By using a good depth map or even better by using the disparity transformation algorithm that mention in this [paper](https://ieeexplore.ieee.org/abstract/document/8890001) and merg it with the original RGB image, the edge will be sharber and more closer to the real pothole edges.

---

Now let's see the depth estimation map on the potholes 600 dataset...

In [ ]:
pothole600_depth_generator = DataGenerator(pothole600_test, 'depth', batch_size=80, shuffle=False, dim=(320,320))
pothole600_depth_images,pothole600_depth_labels = next(iter(pothole600_depth_generator))
pothole600_depth_preds = depth_model.predict(np.array(pothole600_depth_images))

3/3 [==============================] - 172s 51s/step


In [ ]:
plot_image_mask(pothole600_depth_images, pothole600_depth_preds, pothole600_depth_labels)

Output hidden; open in https://colab.research.google.com to view.

The depth estimation model was able to identify the average depth colors, whether they were dark or light. Since the predicted depth images don't have color gradations, I think this will give us a rough idea of the pothole depth, but it won't be exact.

# Using the mask model predictions and the depth model predictions to reconstruct the 3d scene of the pothole

In [9]:
def plot_3d_prediction(depth, mask):
  # img_org = cv2.imread(depth_pred[20]*255)
  img_org = depth
  img_org = cv2.cvtColor(img_org, cv2.COLOR_GRAY2BGR)

  img_mask_gray = mask
  # img_mask = cv2.cvtColor(img_mask, cv2.COLOR_BGR2GRAY)
  img_mask = cv2.cvtColor(img_mask_gray, cv2.COLOR_GRAY2BGR)
  ##Resizing images
  img_org = cv2.resize(img_org, (400,400), interpolation = cv2.INTER_AREA)
  img_mask = cv2.resize(img_mask, (400,400), interpolation = cv2.INTER_AREA)

  array = np.array(img_mask_gray, np.uint8)
  print(array.shape)

  (cnt, hierarchy) = cv2.findContours(array, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  print(f'There are {len(cnt)} pothole(s) in the image')
  img_org = cv2.bitwise_and(img_mask, img_org)
  img_org = img_org*255


  y = range( img_org.shape[0] )
  x = range( img_org.shape[1] ) 
  X, Y = np.meshgrid(x, y)

  Z = img_org[:,:,0]
  print('Predicted depth map + predicted mask: \n\n')
  print('Estimated Width In px:', sum((mask*255 > 1).any(axis=0))[0])
  print('Estimated Height In px:', sum((mask *255> 1).any(axis=1))[0])
  print('Estimated Depth In px: ', round(Z.max() - Z.min(), 2))
  print('Estimated Volume In px:', round(sum((mask*255 > 1).any(axis=0))[0] * sum((mask *255> 1).any(axis=1))[0] *  Z.max() - Z.min(), 2))

  new_x = np.zeros((400, 400))
  new_z = np.zeros((400, 400))
  new_y = np.zeros((400, 400))


  for i in range(400):
    for j in range(400):
      if Z[i, j] < 1 :
        new_x[i, j]=np.NaN
        new_z[i, j] = np.NaN
        new_y[i, j] = np.NaN
      else:
        new_x[i, j]=X[i, j]
        new_z[i, j] = Z[i, j]
        new_y[i, j] = Y[i, j]

  # fig = go.Figure(data=[go.Surface(z=-new_z, x=-new_x, y=-new_y)])
  # fig.update_layout(title='3-D Scene Of The Pothole', autosize=False,
  #                   width=500, height=500,
  #                   margin=dict(l=65, r=50, b=65, t=90))
  # fig.show()
  return new_x, new_z, new_y

In [10]:
def plot_3d_true(i):
  img_org = cv2.imread(pothole600_test.depth[int(i)])

  img_mask = cv2.imread(pothole600_test['mask'][int(i)])
  ##Resizing images
  img_org = cv2.resize(img_org, (400,400), interpolation = cv2.INTER_AREA)
  img_mask = cv2.resize(img_mask, (400,400), interpolation = cv2.INTER_AREA)
  
  # array = np.array(cv2.cvtColor(img_mask, cv2.COLOR_BGR2GRAY), np.uint8)
  # print(array.shape)
  # (cnt, hierarchy) = cv2.findContours(array, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  # print(f'There are {len(cnt)} pothole(s) in the image')

  img_org = cv2.bitwise_and(img_mask, img_org)
  
  y = range( img_org.shape[0] )
  x = range( img_org.shape[1] ) 
  X, Y = np.meshgrid(x, y)

  Z = img_org[:,:,0]
  print('True depth map + true mask: \n\n')
  print('Estimated Width In px:', sum((img_mask>0).any(axis=0))[0])
  print('Estimated Height In px:', sum((img_mask >0).any(axis=1))[0])
  print('Estimated Depth In px: ', round(Z.max() - Z.min(), 2))
  print('Estimated Volume In px:', round(sum((img_mask > 0).any(axis=0))[0] * sum((img_mask > 0).any(axis=1))[0] *  Z.max() - Z.min(), 2))

  new_x = np.zeros((400, 400))
  new_z = np.zeros((400, 400))
  new_y = np.zeros((400, 400))


  for i in range(400):
    for j in range(400):
      if Z[i, j] ==0 :
        new_x[i, j]=np.NaN
        new_z[i, j] = np.NaN
        new_y[i, j] = np.NaN
      else:
        new_x[i, j]=X[i, j]
        new_z[i, j] = Z[i, j]
        new_y[i, j] = Y[i, j]

  # fig = go.Figure(data=[go.Surface(z=-new_z, x=-new_x, y=-new_y)])
  # fig.update_layout(title='3-D Scene Of The Pothole', autosize=False,
  #                   width=500, height=500,
  #                   margin=dict(l=65, r=50, b=65, t=90))
  # fig.show()
  return new_x, new_z, new_y

In [11]:
def compare_3d_reconstruct(i):
  pred_x, pred_z, pred_y = plot_3d_prediction(pothole600_depth_preds[int(i)], pothole600_preds[int(i)])
  print('==============================')
  true_x, true_z, true_y = plot_3d_true(int(i))

  # Initialize figure with 4 3D subplots
  fig = make_subplots(
      rows=1, cols=2,
      specs=[[{'type': 'surface'}, {'type': 'surface'}]])


  fig.add_trace(
      go.Surface(x=-true_x, y=-true_y, z=-true_z, colorscale='Viridis', showscale=False),
      row=1, col=1)



  fig.add_trace(
      go.Surface(x=-pred_x, y=-pred_y, z=-pred_z, colorscale='YlOrRd', showscale=False),
      row=1, col=2)


  fig.update_layout(
      title_text='3D scene of the pothole, true(left) and predicted(right)')

  fig.show()

In [12]:
compare_3d_reconstruct(0)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
compare_3d_reconstruct(79)

Output hidden; open in https://colab.research.google.com to view.

In [13]:
compare_3d_reconstruct(7)

Output hidden; open in https://colab.research.google.com to view.

In [14]:
compare_3d_reconstruct(4)

Output hidden; open in https://colab.research.google.com to view.

In [15]:
compare_3d_reconstruct(30)

Output hidden; open in https://colab.research.google.com to view.

In [16]:
compare_3d_reconstruct(63)

Output hidden; open in https://colab.research.google.com to view.

In [17]:
compare_3d_reconstruct(9)

Output hidden; open in https://colab.research.google.com to view.

In [18]:
compare_3d_reconstruct(33)

Output hidden; open in https://colab.research.google.com to view.